In [1]:
import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

import numpy as np
import math
import os
os.makedirs("images", exist_ok=True)

In [2]:
n_epochs = int(200)
batch_size = int(64)
lr = float(0.0002)
b1 = float(0.5)
b2 = float(0.999)
n_cpu = int(8)
latent_dim = int(100)
img_size = int(32)
channels = int(1)
sample_interval = int(400)

cuda = True if torch.cuda.is_available() else False

# m = model
def weight_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

class DCGAN_GEN(nn.Module):
    def __init__(self):
        super(DCGAN_GEN, self).__init__()
        
        self.init_size = img_size // 4
        # Sequential은 input을 넣을 모델.
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size ** 2))
        
        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )
        
    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

    
class DCGAN_DIS(nn.Module):
    def __init__(self):
        super(DCGAN_DIS, self).__init__()
        
        def discriminator_block(in_filters, out_filters, bn=True):
            # 레이어 3개 list로 묶음
            block = [nn.Conv2d(in_filters, out_filters, 3,2,1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block
        
        self.model = nn.Sequential(
            *discriminator_block(channels, 16, bn=True),
            *discriminator_block(16,32),
            *discriminator_block(32,64),
            *discriminator_block(64,128),
        )
        
        ds_size = img_size// 2**4
        self.adv_layer = nn.Sequential(nn.Linear(128*ds_size**2,1), nn.Sigmoid())
    
    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)
        return validity    

In [7]:
# loss
adversarial_loss = torch.nn.BCELoss()

# init generator/discriminator
dcgan_gen = DCGAN_GEN()
dcgan_dis = DCGAN_DIS()

if cuda :
    dcgan_gen.cuda()
    dcgan_dis.cuda()
    adversarial_loss.cuda()

In [10]:
# init weight
dcgan_gen.apply(weight_init_normal)
dcgan_dis.apply(weight_init_normal)

# optimizer
optimizer_g = torch.optim.Adam(dcgan_gen.parameters(), lr=lr, betas=(b1,b2))
optimizer_d = torch.optim.Adam(dcgan_dis.parameters(), lr=lr, betas=(b1,b2))

# 쿠다쓴다면 쿠다텐서로 변환
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [11]:
optimizer_d

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.5, 0.999)
    eps: 1e-08
    lr: 0.0002
    weight_decay: 0
)

In [14]:
# dataset
os.makedirs("../../data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../../data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=batch_size,
    shuffle=True,
)

In [18]:
# train
for epoch in range(n_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)
            
        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_g.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))

        # Generate a batch of images
        # 제너레이터의 이미지 생성
        gen_imgs = dcgan_gen(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(dcgan_dis(gen_imgs), valid)
        # backward??
        g_loss.backward()
        
        # setp이란게 훈련을 의미 (=train_on_batch)
        optimizer_g.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------
        
        optimizer_d.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(dcgan_dis(real_imgs), valid)
        fake_loss = adversarial_loss(dcgan_dis(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_d.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(dataloader) + i
        
        if batches_done % sample_interval == 0:
            save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)

[Epoch 0/200] [Batch 0/938] [D loss: 0.693212] [G loss: 0.707333]
[Epoch 0/200] [Batch 1/938] [D loss: 0.693196] [G loss: 0.706614]
[Epoch 0/200] [Batch 2/938] [D loss: 0.693108] [G loss: 0.706052]
[Epoch 0/200] [Batch 3/938] [D loss: 0.693046] [G loss: 0.705474]
[Epoch 0/200] [Batch 4/938] [D loss: 0.692981] [G loss: 0.704694]
[Epoch 0/200] [Batch 5/938] [D loss: 0.692844] [G loss: 0.704082]
[Epoch 0/200] [Batch 6/938] [D loss: 0.692798] [G loss: 0.703571]
[Epoch 0/200] [Batch 7/938] [D loss: 0.692557] [G loss: 0.702968]
[Epoch 0/200] [Batch 8/938] [D loss: 0.692457] [G loss: 0.702341]
[Epoch 0/200] [Batch 9/938] [D loss: 0.692358] [G loss: 0.701683]
[Epoch 0/200] [Batch 10/938] [D loss: 0.692084] [G loss: 0.701020]
[Epoch 0/200] [Batch 11/938] [D loss: 0.691784] [G loss: 0.699975]
[Epoch 0/200] [Batch 12/938] [D loss: 0.691406] [G loss: 0.699244]
[Epoch 0/200] [Batch 13/938] [D loss: 0.691788] [G loss: 0.698134]
[Epoch 0/200] [Batch 14/938] [D loss: 0.692043] [G loss: 0.696036]


KeyboardInterrupt: 